### Importing Necessary Libraries

In [ ]:
import cv2
from pylab import array
import sys
from matplotlib import pyplot as plt
import numpy as np

### Network Initialization

COCO Output Format Nose – 0, Neck – 1, Right Shoulder – 2, Right Elbow – 3, Right Wrist – 4, Left Shoulder – 5, Left Elbow – 6, Left Wrist – 7, Right Hip – 8, Right Knee – 9, Right Ankle – 10, Left Hip – 11, Left Knee – 12, LAnkle – 13, Right Eye – 14, Left Eye – 15, Right Ear – 16, Left Ear – 17, Background – 18

MPII Output Format Head – 0, Neck – 1, Right Shoulder – 2, Right Elbow – 3, Right Wrist – 4, Left Shoulder – 5, Left Elbow – 6, Left Wrist – 7, Right Hip – 8, Right Knee – 9, Right Ankle – 10, Left Hip – 11, Left Knee – 12, Left Ankle – 13, Chest – 14, Background – 15

In [ ]:
MODE = "MPI"

if MODE is "COCO":
    protoFile = "models/pose/coco/pose_deploy_linevec.prototxt"
    weightsFile = "models/pose/coco/pose_iter_440000.caffemodel"
    nPoints = 18
    POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

elif MODE is "MPI" :
    protoFile = "models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
    weightsFile = "models/pose/mpi/pose_iter_160000.caffemodel"
    nPoints = 15
    POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
    body_parts = {
        0: "Head",
        1: "Neck",
        2: "Right Shoulder",
        3: "Right Elbow",
        4: "Right Wrist",
        5: "Left Shoulder",
        6: "Left Elbow",
        7: "Left Wrist",
        8: "Right Hip",
        9: "Right Knee",
        10: "Right Ankle",
        11: "Left Hip",
        12: "Left Knee",
        13: "Left Ankle",
        14: "Chest",
        15: "Background"
    }
    weight_distrib = {
        0: 0.07/2,
        1: 0.07/2,
        2: 0.13/6,
        3: 0.13/6,
        4: 0.13/6,
        5: 0.13/6,
        6: 0.13/6,
        7: 0.13/6,
        8: 0.43/3,
        9: 0.37/4,
        10: 0.37/4,
        11: 0.43/3,
        12: 0.37/4,
        13: 0.37/4,
        14: 0.43/3,
        15: "Background"
    }

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
net.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
print("Using CPU device")

### Testing camera

In [ ]:
# Testing camera
cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)

# cap.set(cv2.CAP_PROP_EXPOSURE, -8.0)
print(cap)

while(True):

    # Capture the video frame
    # by frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)

    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()

### Reading in the frames

In [134]:
import time
time.sleep(4)
# Initial video reading
FRAMES_READ = 60  # Assume 30 fps
FRAMES = []

cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)  # Apparently dshow makes it faster
for i in range(FRAMES_READ):
    ret, frame = cap.read()
    FRAMES.append(frame)
    cv2.imshow("frame", frame)
    cv2.setWindowProperty("frame", cv2.WND_PROP_TOPMOST, 1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()
print(len(FRAMES), array(FRAMES).nbytes/(1024*1024))

60 52.734375


### Looping the video

In [135]:
# for i in range(10):
# for j in loop[:5]:
    # print(j)
# print(loop)
# import time
for _ in range(1):
    for frame in FRAMES:
        cv2.imshow("frame", frame)
        cv2.waitKey(0)
        # cv2.setWindowProperty("frame", cv2.WND_PROP_TOPMOST, 1)
        print("bruh")
        # time.sleep(2)
        # cv2.imshow("frame", loop)
        # print("bruh2")

cv2.waitKey(0)
cv2.destroyAllWindows()

bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh


### Pose Processing

In [145]:
loop = []
center_of_grav_loc = []

for frame in FRAMES:
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    threshold = 0.1

    inWidth = 250
    inHeight = 250
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                            (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    # Bottleneck

    output = net.forward()
    H = output.shape[2]
    W = output.shape[3]

    # Empty list to store the detected keypoints
    points = []

    x_sum = 0
    y_sum = 0

    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # Scale the point to fit on the original image
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        x_sum += x*weight_distrib[i]
        y_sum += y*weight_distrib[i]

        if prob > threshold or 1:
            cv2.circle(frameCopy, (int(x), int(y)), 4, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, f"{i}: {body_parts[i]}", (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else :
            points.append(None)
    
    cv2.circle(frameCopy, (int(x_sum), int(y_sum)), 10, (0, 255, 0), thickness=-1, lineType=cv2.FILLED)
    center_of_grav_loc.append((int(x_sum), int(y_sum)))

    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        # if points[partA] and points[partB]:
        #     cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
        #     cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

    loop.append(frameCopy)
    # cv2.imshow('Output-Keypoints', frameCopy)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break
    print("processing...")
print("done")



processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
done


### Looping the image

In [148]:
# for i in range(10):
# for j in loop[:5]:
    # print(j)
# print(loop)
# import time
for _ in range(1):
    for frame in loop:
        cv2.imshow("frame", frame)
        cv2.waitKey(0)
        # cv2.setWindowProperty("frame", cv2.WND_PROP_TOPMOST, 1)
        print("bruh")
        # time.sleep(2)
        # cv2.imshow("frame", loop)
        # print("bruh2")

cv2.waitKey(0)
cv2.destroyAllWindows()

bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh
bruh


### COG Distance

In [146]:
tot = 0
for i, j in zip(center_of_grav_loc[:-1], center_of_grav_loc[1:]):
    dist = ((j[0]-i[0])**2 + (j[1]-i[1])**2)**0.5
    tot += dist
    print(f"{i} -> {j}: {dist} pixels")
print(tot, "pixels")

(287, 235) -> (279, 230): 9.433981132056603 pixels
(279, 230) -> (276, 230): 3.0 pixels
(276, 230) -> (281, 230): 5.0 pixels
(281, 230) -> (281, 230): 0.0 pixels
(281, 230) -> (281, 230): 0.0 pixels
(281, 230) -> (281, 230): 0.0 pixels
(281, 230) -> (279, 232): 2.8284271247461903 pixels
(279, 232) -> (280, 230): 2.23606797749979 pixels
(280, 230) -> (279, 232): 2.23606797749979 pixels
(279, 232) -> (276, 232): 3.0 pixels
(276, 232) -> (274, 232): 2.0 pixels
(274, 232) -> (276, 232): 2.0 pixels
(276, 232) -> (274, 232): 2.0 pixels
(274, 232) -> (276, 232): 2.0 pixels
(276, 232) -> (281, 232): 5.0 pixels
(281, 232) -> (281, 230): 2.0 pixels
(281, 230) -> (281, 230): 0.0 pixels
(281, 230) -> (279, 230): 2.0 pixels
(279, 230) -> (277, 230): 2.0 pixels
(277, 230) -> (283, 230): 6.0 pixels
(283, 230) -> (281, 230): 2.0 pixels
(281, 230) -> (284, 230): 3.0 pixels
(284, 230) -> (284, 229): 1.0 pixels
(284, 229) -> (281, 228): 3.1622776601683795 pixels
(281, 228) -> (283, 230): 2.82842712474619